# Concepts Tutorial

SymForce uses [concepts](https://en.wikipedia.org/wiki/Concept_(generic_programming%29)) as an underlying mechanism. A concept is a specification of supported operations, including syntax and semantics, but does not require a subtype relationship. This means that a set of heterogenous types can be operated on in a homogenous way, i.e. types that are external and don't share a base class, like Python floats treated as scalars.

There are three core concepts, each of which is a superset of the previous. The core routines use these ops interfaces rather than calling methods on types directly. The [ops package](../api/symforce.ops.html) docs provide much more detail and each op is tested on each type, but examples are given here.

In the case of lists of objects or a Values object (see the [Values tutorial](../notebooks/values_tutorial.html)), we perform element-wise operations.

In [ ]:
# Setup
import symforce
symforce.set_backend('sympy')
symforce.set_log_level('warning')

from symforce.notebook_util import display
from symforce import geo
from symforce import sympy as sm
from symforce.values import Values
from symforce.ops import StorageOps, GroupOps, LieGroupOps

## StorageOps

[StorageOps](../api/symforce.ops.storage_ops.html): **Data type that can be serialized to and from a vector of scalar quantities.**

Methods: `.storage_dim()`, `.to_storage()`, `.from_storage()`, `.symbolic()`, `.evalf()`, `.subs()`, `.simplify()`

Storage operations are used extensively for marshalling and for operating on each scalar in a type.

In [ ]:
# Number of scalars used to represent a Pose3 (4 quaternion + 3 position)
display(StorageOps.storage_dim(geo.Pose3))

In [ ]:
# Because we are using concepts, we can operate on types that aren't subtypes of symforce
display(StorageOps.storage_dim(float))

In [ ]:
# Element-wise operations on lists of objects
display(StorageOps.storage_dim([geo.Pose3, geo.Pose3]))

In [ ]:
# Element-wise operations on Values object with multiple types of elements
values = Values(
    pose=geo.Pose3(),
    scalar=sm.Symbol('x')
)
display(StorageOps.storage_dim(values))  # 4 quaternion + 3 position + 1 scalar

In [ ]:
# Serialize scalar
display(StorageOps.to_storage(5))

In [ ]:
# Serialize vector/matrix
display(StorageOps.to_storage(geo.V3(sm.Symbol('x'), 5.2, sm.sqrt(5))))

In [ ]:
# Serialize geometric type and reconstruct
T = geo.Pose3.symbolic('T')
T_serialized = StorageOps.to_storage(T)
T_recovered = StorageOps.from_storage(geo.Pose3, T_serialized)
display(T_serialized)
display(T_recovered)

## GroupOps

[GroupOps](../api/symforce.ops.group_ops.html): **Mathematical group that implements closure, associativity, identity and invertibility.**

Methods: `.identity()`, `.inverse()`, `.compose()`, `.between()`

Group operations provide the core methods to compare and combine types.

In [ ]:
# Identity of a pose
display(GroupOps.identity(geo.Pose3))

In [ ]:
# Identity of a scalar (under addition)
display(GroupOps.identity(float))

In [ ]:
# Inverse of a vector
display(GroupOps.inverse(geo.V3(1.2, -3, 2)).T)

In [ ]:
# Compose two vectors (under addition)
display(GroupOps.compose(geo.V2(1, 2), geo.V2(3, -5)))

In [ ]:
# Compose a rotation and its inverse to get identity
R1 = geo.Rot3.from_angle_axis(
    angle=sm.Symbol('theta1'),
    axis=geo.V3(0, 0, 1)
)
display(StorageOps.simplify(GroupOps.compose(R1, R1.inverse()).simplify()))

In [ ]:
# Relative rotation using `.between()`
R2 = geo.Rot3.from_angle_axis(
    angle=sm.Symbol('theta2'),
    axis=geo.V3(0, 0, 1)
)
R_delta = GroupOps.between(R1, R2)
display(R2)
display(StorageOps.simplify(GroupOps.compose(R1, R_delta)))

## LieGroupOps

[LieGroupOps](../api/symforce.ops.lie_group_ops.html): **Group that is also a differentiable manifold, such that calculus applies.**

Methods: `.tangent_dim()`, `.from_tangent()`, `to_tangent()`, `.retract()`, `.local_coordinates()`, `.storage_D_tangent()`

Lie group operations provide the core methods for nonlinear optimization.  
Familiarity is not expected for all users, but learning is encouraged!

In [ ]:
# Underlying dimension of a 3D rotation's tangent space
display(LieGroupOps.tangent_dim(geo.Rot3))

In [ ]:
# Exponential map (tangent space vector wrt identity element) for a 2D rotation
angle = sm.Symbol('theta')
rot2 = LieGroupOps.from_tangent(geo.Rot2, [angle])
display(rot2.to_rotation_matrix())

In [ ]:
# Logarithmic map (tangent space wrt identity element -> element) of the rotation
display(LieGroupOps.to_tangent(rot2))

In [ ]:
# Exponential map of a vector type is a no-op
display(LieGroupOps.from_tangent(geo.V5(), [1, 2, 3, 4, 5]).T)

In [ ]:
# Retract perturbs the given element in the tangent space and returns the updated element
rot2_perturbed = LieGroupOps.retract(rot2, [sm.Symbol('delta')])
display(rot2_perturbed.to_rotation_matrix())

In [ ]:
# Local coordinates compute the tangent space perturbation between one element and another
display(StorageOps.simplify(LieGroupOps.local_coordinates(rot2, rot2_perturbed)))

In [ ]:
# storage_D_tangent computes the jacobian of the storage space of an object with respect to
# the tangent space around the element.

# A 2D rotation is represented by a complex number, so storage_D_tangent represents how
# that complex number will change given an infinitesimal perturbation in the tangent space
display(LieGroupOps.storage_D_tangent(rot2))

Using symbolic geometric types and concepts is already very powerful for development and analysis of robotics, but operating on symbolic objects at runtime is much too slow for most applications. However, symbolic expressions can be beautifully set to fast runtime code.